In [20]:
import tensorflow as tf
import numpy as np
import time
import tempfile

In [21]:
data = np.genfromtxt('data/train.csv', delimiter=',', dtype=np.float32)
train_images = data[1:42001,1:785]
data = data[1:42001,0]
train_labels = np.ndarray((42000,10))
for i in range(42000):
    for j in range(10):
        if data[i] == j:
            train_labels[i,j]=1
            imageDefinition = len(train_images[0])

In [22]:
sess = tf.InteractiveSession()
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial, name=name)

In [23]:
def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

In [24]:
def conv_layer(input, channels_in, channels_out, filter_size, name="conv"):
    with tf.name_scope(name):
        w_conv = weight_variable([filter_size, filter_size, channels_in, channels_out], "filters")
        b_conv = bias_variable([channels_out], "biases")
        conv = tf.nn.conv2d(input, w_conv, strides=[1, 1, 1, 1], padding="SAME")
        conv_act = tf.nn.relu(conv + b_conv)
        tf.summary.histogram("filters", w_conv)
        tf.summary.histogram("biases", b_conv)
        tf.summary.histogram("activations", conv_act)
        conv_act = tf.nn.max_pool(conv_act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
        image = conv_act[0:1,:,:,0:channels_out]
        image = tf.transpose(image, perm=[3, 1, 2, 0])
        tf.summary.image(name, image, 3)
        return conv_act

In [25]:
def fc_layer(input, channels_in, channels_out, name="fc"):
    with tf.name_scope(name):
        w = weight_variable([channels_in, channels_out], name="W")
        b = bias_variable([channels_out,], "B")
        act = tf.nn.relu(tf.matmul(input, w)+b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [26]:
numClass = len(train_labels[0])
num_filter = 40
num_features = 8*num_filter*7*7
numNeuron = 5000
learning_rate = 1e-6
dropout_rate = 0.4
batchSize = 100
numStep = 20000

In [27]:
images_vec = tf.placeholder(dtype=tf.float32, shape=(None, imageDefinition), name="images")
labels = tf.placeholder(dtype=tf.float32, shape=(None, numClass), name="labels")

In [28]:
images_mat = tf.reshape(images_vec, [-1, 28, 28, 1])
tf.summary.image('input', images_mat, 9)

<tf.Tensor 'input_1:0' shape=() dtype=string>

In [29]:
images_pooled = tf.nn.max_pool(tf.nn.max_pool(images_mat, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME"), ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [30]:
conv3x3_1 = conv_layer(images_mat, 1, num_filter, 3, "conv3x3_1")
conv4x4_1 = conv_layer(images_mat, 1, num_filter, 4, "conv4x4_1")
conv5x5_1 = conv_layer(images_mat, 1, num_filter, 5, "conv5x5_1")
conv6x6_1 = conv_layer(images_mat, 1, num_filter, 6, "conv6x6_1")

In [31]:
conv_1 = tf.concat([conv3x3_1, conv4x4_1, conv5x5_1, conv6x6_1, ], 3)

In [32]:
conv3x3_2 = conv_layer(conv_1, 4*num_filter, 8*num_filter, 3, "conv3x3_2")
conv4x4_2 = conv_layer(conv_1, 4*num_filter, 8*num_filter, 4, "conv4x4_2")
conv5x5_2 = conv_layer(conv_1, 4*num_filter, 8*num_filter, 5, "conv5x5_2") 
conv6x6_2 = conv_layer(conv_1, 4*num_filter, 8*num_filter, 6, "conv6x6_2")

In [33]:
conv_2 = tf.concat([tf.reshape(conv3x3_2, [-1, num_features]), tf.reshape(conv4x4_2, [-1, num_features]), tf.reshape(conv5x5_2, [-1, num_features]), tf.reshape(conv6x6_2, [-1, num_features]), tf.reshape(images_pooled, [-1, 7*7])], 1)

In [34]:
fc1 = fc_layer(conv_2, num_features*4+7*7, numNeuron, name="fc1")
fc2 = fc_layer(fc1, numNeuron, numNeuron, name="fc2")
keep_prob = tf.placeholder(tf.float32)
fc2_dropout = tf.nn.dropout(fc2, keep_prob)
logits = fc_layer(fc2_dropout, numNeuron, numClass, name="logits")

In [35]:
with tf.name_scope("xent"):
    xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
    tf.summary.scalar('cross_entropy', xent)

In [36]:
with tf.name_scope("train"):
    train_step=tf.train.AdamOptimizer(learning_rate).minimize(xent)

In [37]:
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction, tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

In [38]:
merged_summary = tf.summary.merge_all()
graph_location = tempfile.mkdtemp()
print("---------------------------------------------------------") 
print("----- graph available here : ", graph_location)
print("---------------------------------------------------------")
train_writer = tf.summary.FileWriter(graph_location, sess.graph)
tf.global_variables_initializer().run()

---------------------------------------------------------
----- graph available here :  C:\Users\Steven\AppData\Local\Temp\tmp85n3znbb
---------------------------------------------------------


ResourceExhaustedError: OOM when allocating tensor of shape [5,5,160,320] and type float
	 [[Node: conv5x5_2/filters/Adam/Initializer/zeros = Const[_class=["loc:@conv5x5_2/filters"], dtype=DT_FLOAT, value=Tensor<type: float shape: [5,5,160,320] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'conv5x5_2/filters/Adam/Initializer/zeros', defined at:
  File "e:\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "e:\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "e:\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "e:\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "e:\python\python36\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "e:\python\python36\lib\site-packages

In [ ]:
batchIndex = np.arange(0, 41999, batchSize)
ones = np.ones(100000, dtype=int)
batchIndex = np.kron(ones,batchIndex)
batchIndex = batchIndex[0:numStep]
j = 0

In [ ]:
timeStart = time.time()
for i in batchIndex:
    j = j + 1
    sess.run(train_step, feed_dict={images_vec: train_images[i:i+batchSize, ], labels: train_labels[i:i+batchSize, ], keep_prob: dropout_rate})
    if j % 100 == 0:
        s = sess.run(merged_summary, feed_dict={images_vec: train_images[i:i+batchSize, ], labels: train_labels[i:i+batchSize, ], keep_prob: 1.0}) train_writer.add_summary(s, j) train_accuracy = accuracy.eval(feed_dict={images_vec: train_images[i:i+batchSize, ], labels: train_labels[i:i+batchSize, ], keep_prob: 1.0}) print(" %g%% completed, training accuracy %g%%" % (j/numStep*100.00, train_accuracy*100.00)) print("total time for training : ", (time.time()-timeStart)/60, " minutes.")